# Assignment 2
# Ling Fei Zhang, 260985358
# Brandon Ma, ID

## Question 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from functions import SARSA, visualize
import time
import gymnasium as gym


c:\Users\Ling0\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here, we will set a few parameters for our environment.

In [2]:
ALPHAS = [0.01, 0.1, 0.2]
GAMMA = 0.9
TEMPERATURE = [1, 10, 100]
EPISODES = 5500
RUNS = 10


Driver Code to test out the combination of each alpha with each temperature for SARSA.

In [3]:
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False)
env.reset()
sarsa = SARSA(env, 0.1, 1, GAMMA, EPISODES, False)
# print(type(env.action_space.n))
rewards = sarsa.simulate_episodes()
final_policy = sarsa.learned_policy
print("final policy: ", final_policy.reshape(4,4))
# print("total reward: ", rewards)

#uncomment this line if you want to vizualize the game
# visualize(final_policy, 10)

100%|██████████| 5500/5500 [00:01<00:00, 4238.90it/s]

final policy:  [[0. 2. 1. 0.]
 [1. 3. 1. 2.]
 [2. 2. 1. 3.]
 [2. 2. 2. 1.]]
total reward:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.058823529411764705, 0.0, 0.0, 0.0, 0.0,

In [22]:
for alpha in ALPHAS:
    for temp in TEMPERATURE:
        env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False)
        env.reset()
        sarsa = SARSA(env, alpha, temp, GAMMA, EPISODES, False)

### Effect of parameters on final training performance for SARSA

### Effect of parameters on final testing performance for SARSA

### Best parameter for SARSA

### Conclusion for SARSA

### Effect of parameters on final training performance for EXPECTED SARSA

### Effect of parameters on final testing performance for EXPECTED SARSA

### Best parameter for EXPECTED SARSA

### Conclusion for EXPECTED SARSA